In [1]:
import random

##### First modification

In [ ]:
def recreate_min_time(instance, solution, w1, w2, w3, w4, w5, gamma):
    solution[1] = sort_absent_customers(instance, solution, w1, w2, w3, w4, w5)
    for c in solution[1]:
        pos_best = None
        makespan_best = float('inf')
        
        feasible_truck_tour_positions = feasible_truck_tour_positions_calculation(instance, solution[0][0], c)
        for pos in feasible_truck_tour_positions: 
            makespan_diff = completion_time(instance, insert_customer(c, pos, solution)) - makespan(instance, solution)
            if pos_best == None or ((pos_best != None and makespan_diff <= makespan_best) and random.random() > (1 - gamma)):
                makespan_best = makespan_diff
                pos_best = pos
        
        if drone_eligible(isntance, solution[0][1], c):
            chosen_drone = largest_time_slack_drone(solution[0][1])
            makespan_diff = completion_time(instance, insert_customer(c, [1, chosen_drone, 0], solution)) - makespan(instance, solution)
            if pos_best == None or ((pos_best != None and makespan_diff <= makespan_best) and random.random() > (1 - gamma)):
                makespan_best = makespan_diff
                pos_best = [1, chosen_drone, 0]

        if pos_best == None:
            solution[0][0].append([])
            pos_best[0, len(solution[0][0]) - 1, 0]
        
        solution = insert_customer(c, pos_best, solution)
        solution[1].remove(c)

    return solution

##### Second modification

In [ ]:
def local_search_min_time(instance, solution, n_nearest):
    interrupt_truck_tours = False
    interrupt_truck_customers = False

    interrupt_drone_tours = False
    #interrupt_drone_customers = False


    for i in range(len(solution[0][0])):
        if interrupt_truck_tours:
            break
        for customer in range(len(solution[0][0][i])):
            if interrupt_truck_customers:
                interrupt_truck_tours = True
                break
            neighbors = select_nearest_neighbors(n_nearest, instance.distances[customer])
            for neighbor in neighbors:
                if any(neighbor in truck_tour for truck_tour in solution[0][0]):
                    neighbor_index = next((j for j, node in enumerate(solution[0][0][i]) if node == neighbor), None)
                    truck_tour_neighbor = find_customer_in_vehicle_tours(solution[0][0], neighbor)
                    t = max(truck_tour_time(instance.t_t, solution[0][0][i]), truck_tour_time(instance.t_t, solution[0][0][truck_tour_neighbor]))

                    if neighbor_index == None:
                        new_solution = _2_opt_x(instance, solution.copy(), customer, neighbor, i)
                        if ((truck_tur_time(instance.t_t, new_solution[0][0][i]) <= t and truck_tur_time(instance.t_t, new_solution[0][0][truck_tour_neighbor]) < t) or (truck_tur_time(instance.t_t, new_solution[0][0][i]) < t and truck_tur_time(instance.t_t, new_solution[0][0][truck_tour_neighbor]) <= t)):
                            solution = local_search(instance, new_solution, n_nearest)
                            interrupt_truck_customers = True
                            break
                    else: 
                        new_solution = relocate(instance, solution.copy(), customer, neighbor, i)
                        if truck_tour_time(instance, new_solution[0][0][i]) < truck_tour_time(instance, solution[0][0][i]):
                            solution = local_search(instance, new_solution, n_nearest)
                            interrupt_truck_customers = True
                            break

                        new_solution = swap(solution.copy(), customer, neighbor, i)
                        if truck_tour_time(instance, new_solution[0][0][i]) < truck_tour_time(instance, solution[0][0][i]):
                            solution = local_search(instance, new_solution, n_nearest)
                            interrupt_truck_customers = True
                            break

                        new_solution = _2_opt(solution.copy(), customer, neighbor, i)
                        if truck_tour_time(instance, new_solution[0][0][i]) < truck_tour_time(instance, solution[0][0][i]):
                            solution = local_search(instance, new_solution, n_nearest)
                            interrupt_truck_customers = True
                            break

                else:
                    drone_tour_neightbor = find_customer_in_vehicle_tours(solution[0][0], neighbor)
                    t = max(truck_tour_time(instance.t_t, solution[0][0][i]), drone_tour_time(instance.t_t, solution[0][1][drone_tour_neightbor]))

                    new_solution = swap_x(instance, solution.copy(), customer, neighbor, i)
                    if ((truck_tour_time(instance.t_t, new_solution[0][0][i]) <= t and drone_tour_time(instance.t_d, new_solution[0][1][drone_tour_neightbor]) < t) or (truck_tour_time(instance.t_t, new_solution[0][0][i]) < t and drone_tour_time(instance.t_d, new_solution[0][1][drone_tour_neightbor]) <= t)):
                        solution = local_search(instance, new_solution, n_nearest)
                        interrupt_truck_customers = True
                        break

            chosen_drone = largest_spatial_slack_drone(instance, solution[0][1])
            t = max(truck_tour_time(instance.t_t, solution[0][0][i]), drone_tour_time(instance.t_d, solution[0][1][chosen_drone]))


            new_solution = shift_t(instance, solution, customer, i, chosen_drone)
            if ((truck_tour_time(instance.t_t, new_solution[0][0][i]) <= t and drone_tour_time(instance.t_d, new_solution[0][1][chosen_drone]) < t) or (truck_tour_time(instance.t_t, new_solution[0][0][i]) < t and drone_tour_time(instance.t_d, new_solution[0][1][chosen_drone]) <= t)):
                solution = local_search(instance, new_solution, n_nearest)
                interrupt_truck_tours = True
                break

    for i in range(len(solution[0][1])):
        if interrupt_drone_tours:
            break
        for customer in range(len(solution[0][1][i])):
            new_solution = shift_d(solution, customer, i)
            if drone_tour_time(instance, new_solution[0][1][i]) < drone_tour_time(instance, solution[0][1][i]):
                solution = local_search(instance, new_solution, n_nearest)
                interrupt_drone_tours = True
                break

##### Third modification

In [ ]:
def SISSRs_min_time(instance, delta, epsilon, iter_max):
    s_0 = initial_solution_construction(instance)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    while (terminal_condition_met() == False):
        s = ruin_and_recreate(instance, s_curr)
        if makespan(instance, s) < makespan(instance, s_curr)*(1+delta) or (makespan(instance, s) == makespan(instance, s_curr) and total_completion_time(instance,s) < total_completion_time(instance, s_curr)):
            s_curr = local_search(instance, s)
            if makespan(instance, s_curr) < makespan(instance, s_best) or (makespan(instance, s) == makespan(instance, s_best) and total_completion_time(instance,s) < total_completion_time(instance, s_best)):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_max:
            s_curr = perturbate(instance, s_curr)
            iterations_without_improvement = 0
        delta = delta * epsilon

    return s_best